In [1]:
!miniforge3/bin/mamba activate laplacian2

Run 'mamba init' to be able to run mamba activate/deactivate
and start a new shell session. Or use conda to activate/deactivate.



In [ ]:
!pip3 install cupy numba; # pycuda

In [1]:
!pip3 install combin

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip3 install cupy

Defaulting to user installation because normal site-packages is not writeable
  Using cached cupy-13.1.0.tar.gz (3.5 MB)

In [8]:
from typing import Callable
import numpy as np
import numba as nb
from numba import cuda

In [9]:
# [int(2**k) for k in [3,3.5,4,4.5,5,5.5,6,6.5,7]]
# 16,32,64,128,256,512,1024,2048
[int(2**k) for k in np.arange(4,11,0.5)]

[16, 22, 32, 45, 64, 90, 128, 181, 256, 362, 512, 724, 1024, 1448]

In [ ]:
do_bounds_check = True

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def get_max(top: int, bottom: int, pred: Callable, *args):
  if ~pred(bottom, *args):
    return bottom
  size = (top - bottom)
  while (size > 0):
    step = size >> 1
    mid = top - step
    if ~pred(mid, *args):
      top = mid - 1
      size -= step + 1
    else:
      size = step
  return top

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def find_k_pred(w: int, r: int, m: int, BT: np.ndarray) -> bool:
  return BT[m][w] <= r

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def get_max_vertex(r: int, m: int, n: int, BT: np.ndarray) -> int:
  k_lb: int = m - 1
  return 1 + get_max(n, k_lb, find_k_pred, r, m, BT)

@nb.jit(nopython=True,boundscheck=do_bounds_check)
def k_boundary_cpu(n: int, simplex: int, dim: int, BT: np.ndarray, out: np.ndarray):
  idx_below: int = simplex
  idx_above: int = 0
  j = n - 1
  for k in np.flip(np.arange(dim+1)):
    j = get_max_vertex(idx_below, k + 1, j, BT) - 1
    c = BT[k+1][j]
    face_index = idx_above - c + idx_below
    idx_below -= c
    idx_above += BT[k][j]
    out[dim-k] = face_index

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian0_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(2, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    a,b = ps
    y[a] -= x[b]
    y[b] -= x[a]

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian1_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(4, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    i,j,q,_= ps
    y[i] += (x[q] - x[j])
    y[j] -= (x[j] + x[q])
    y[q] += (x[i] - x[j])

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian2_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(4, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    a,b,c,d = ps
    y[a] += x[c] - (x[b] + x[d])
    y[b] += x[d] - (x[a] + x[c])
    y[c] += x[a] - (x[b] + x[d])
    y[d] += x[b] - (x[a] + x[c])

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian3_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(5, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    a,b,c,d,e = ps
    y[a] += (x[c] + x[e]) - (x[b] + x[d])
    y[b] += x[d] - (x[a] + x[c] + x[e])
    y[c] += (x[a] + x[e]) - (x[b] + x[d])
    y[d] += x[b] - (x[a] + x[c] + x[e])
    y[e] += (x[a] + x[c]) - (x[b] + x[d])

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian4_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(6, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    a,b,c,d,e,f = ps
    y[a] += x[c] + x[e] - (x[b] + x[d] + x[f])
    y[b] += x[d] + x[f] - (x[a] + x[c] + x[e])
    y[c] += x[a] + x[e] - (x[b] + x[d] + x[f])
    y[d] += x[b] + x[f] - (x[a] + x[c] + x[e])
    y[e] += x[a] + x[c] - (x[b] + x[d] + x[f])
    y[f] += x[b] + x[d] - (x[a] + x[c] + x[e])

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def laplacian5_matvec(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  np.multiply(x, deg, y) # y = x * deg
  ps = np.zeros(7, dtype=np.int32)
  for tid in range(N):
    k_boundary_cpu(n, tid, k - 1, BT, ps)
    a,b,c,d,e,f,g = ps
    y[a] += x[c] + x[e] + x[g] - (x[b] + x[d] + x[f])
    y[b] += x[d] + x[f] - (x[a] + x[c] + x[e] + x[g])
    y[c] += x[a] + x[e] + x[g] - (x[b] + x[d] + x[f])
    y[d] += x[b] + x[f] - (x[a] + x[c] + x[e] + x[g])
    y[e] += x[a] + x[c] + x[g] - (x[b] + x[d] + x[f])
    y[f] += x[b] + x[d] - (x[a] + x[c] + x[e] + x[g])
    y[g] += x[a] + x[c] + x[e] - (x[b] + x[d] + x[f])

@nb.jit(nopython=True, boundscheck=do_bounds_check)
def precompute_deg(n: int, k: int, N: int, M: int, BT: np.ndarray) -> np.ndarray:
  deg = np.zeros(N)
  k_faces = np.zeros(k, dtype=np.int32)
  for r in range(M):
    k_boundary_cpu(n, simplex=r, dim=k-1, BT=BT, out=k_faces)
    deg[k_faces] += 1
  return deg

In [ ]:
from combin import rank_to_comb, comb_to_rank
from itertools import combinations
from math import comb
n, k = 8, 3
N, M = comb(n,k-1), comb(n,k)
BT = np.array([[comb(ni, ki) for ni in range(n)] for ki in range(k+2)]).astype(np.int64)
deg = precompute_deg(n,k,N,M,BT)
assert np.all(deg == (np.ones(comb(n,k-1)) * (n - k + 1)))
print(deg)


In [ ]:
from numba import cuda

In [ ]:
import cupy

@cuda.jit(device=True)
# def get_max(top: int, bottom: int, pred: Callable, *args):
def get_max_cuda(top: int, bottom: int, r: int, m: int, BT: np.ndarray) -> int:
  # if ~pred(bottom, *args):
  if not(BT[m][bottom] <= r):
    return bottom
  size = (top - bottom)
  while (size > 0):
    step = size >> 1
    mid = top - step
    if not(BT[m][mid] <= r):
      top = mid - 1
      size -= step + 1
    else:
      size = step
  return top

# @cuda.jit(device=True)
# def find_k_pred(w: int, r: int, m: int, BT: np.ndarray) -> bool:
#   return BT[m][w] <= r

@cuda.jit(device=True)
def get_max_vertex_cuda(r: int, m: int, n: int, BT: np.ndarray) -> int:
  k_lb: int = m - 1
  # return 1 + get_max(n, k_lb, find_k_pred, r, m, BT)
  return 1 + get_max_cuda(n, k_lb, r, m, BT)

@cuda.jit(device=True)
def k_boundary(n: int, simplex: int, dim: int, BT: np.ndarray, out: np.ndarray):
  idx_below: int = simplex
  idx_above: int = 0
  j = n - 1
  for kr in range(dim+1):
    k = dim - kr
    j = get_max_vertex_cuda(idx_below, k + 1, j, BT) - 1
    c = BT[k+1][j]
    face_index = idx_above - c + idx_below
    idx_below -= c
    idx_above += BT[k][j]
    out[kr] = face_index

In [ ]:
from numba import int64, float64

@cuda.jit
def laplacian1_matvec_cuda(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  ps = cuda.local.array(shape=(4,), dtype=int64)

  if tid < N:
    k_boundary(n, tid, k - 1, BT, ps)
    i, j, q = ps[0], ps[1], ps[2]
    cuda.atomic.add(y, i, x[q] - x[j])
    cuda.atomic.add(y, j, -(x[j] + x[q]))
    cuda.atomic.add(y, q, x[i] - x[j])

@cuda.jit
def laplacian2_matvec_cuda(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  tid = (cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x)
  ps = cuda.local.array(shape=(4,), dtype=int64)

  if tid < N:
    k_boundary(n, tid, k - 1, BT, ps)
    a,b,c,d = ps[0], ps[1], ps[2], ps[3]
    cuda.atomic.add(y, a, x[c] - (x[b] + x[d]))
    cuda.atomic.add(y, b, x[d] - (x[a] + x[c]))
    cuda.atomic.add(y, c, x[a] - (x[b] + x[d]))
    cuda.atomic.add(y, d, x[b] - (x[a] + x[c]))


@cuda.jit
def laplacian3_matvec_cuda(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  # cp.multiply(x, deg, y) # y = x * deg
  tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  ps = cuda.local.array(shape=(5,), dtype=int64)
  if tid < N:
    k_boundary(n, tid, k - 1, BT, ps)
    a,b,c,d,e = ps[0], ps[1], ps[2], ps[3], ps[4]
    cuda.atomic.add(y, a, (x[c] + x[e]) - (x[b] + x[d]))
    cuda.atomic.add(y, b, x[d] - (x[a] + x[c] + x[e]))
    cuda.atomic.add(y, c, (x[a] + x[e]) - (x[b] + x[d]))
    cuda.atomic.add(y, d, x[b] - (x[a] + x[c] + x[e]))
    cuda.atomic.add(y, e, (x[a] + x[c]) - (x[b] + x[d]))

@cuda.jit
def laplacian4_matvec_cuda(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  # tid = cuda.grid(1)
  tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  ps = cuda.local.array(shape=(6,), dtype=int64)
  if tid < N:
    k_boundary(n, tid, k - 1, BT, ps)
    a,b,c,d,e,f = ps[0], ps[1], ps[2], ps[3], ps[4], ps[5]
    cuda.atomic.add(y, a, x[c] + x[e] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, b, x[d] + x[f] - (x[a] + x[c] + x[e]))
    cuda.atomic.add(y, c, x[a] + x[e] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, d, x[b] + x[f] - (x[a] + x[c] + x[e]))
    cuda.atomic.add(y, e, x[a] + x[c] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, f, x[b] + x[d] - (x[a] + x[c] + x[e]))

@cuda.jit
def laplacian5_matvec_cuda(x: np.ndarray, y: np.ndarray, n: int, k: int, N: int, BT: np.ndarray, deg: np.ndarray):
  # tid = cuda.grid(1)
  tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  ps = cuda.local.array(shape=(7,), dtype=int64)
  if tid < N:
    k_boundary(n, tid, k - 1, BT, ps)
    a,b,c,d,e,f,g = ps[0], ps[1], ps[2], ps[3], ps[4], ps[5], ps[6]
    cuda.atomic.add(y, a, x[c] + x[e] + x[g] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, b, x[d] + x[f] - (x[a] + x[c] + x[e] + x[g]))
    cuda.atomic.add(y, c, x[a] + x[e] + x[g] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, d, x[b] + x[f] - (x[a] + x[c] + x[e] + x[g]))
    cuda.atomic.add(y, e, x[a] + x[c] + x[g] - (x[b] + x[d] + x[f]))
    cuda.atomic.add(y, f, x[b] + x[d] - (x[a] + x[c] + x[e] + x[g]))
    cuda.atomic.add(y, g, x[a] + x[c] + x[e] - (x[b] + x[d] + x[f]))

# for i in range(len(ps)):
#   if ps[i] < 0 or ps[i] > len(x):
#   # if a < 0 or a > len(x)
#     y[0] = -0.5 # tid #
#     return

In [ ]:
from math import comb
from numba import float32
import cupy as cp

def predict_GB(n,k):
  return 3 * ((comb(n,k-1) * 4) / 1024**3) + (n*(k+2) * 8) / 1024**3

class LaplacianBenchmark():
  def __init__(self, n: int, k: int, gpu: bool = False, threadsperblock: int = 32, n_kernels: int = 1):
    assert k >= 2, "k must be at least 2"
    self.tm = np if not gpu else cp
    self.n = n # num vertices
    self.k = k # dim + 1
    self.N = comb(n,k-1)
    self.M = comb(n,k)
    self._pred_GB = 3 * ((comb(self.n,self.k-1) * 4) / 1024**3) + (self.n*(self.k+2) * 8) / 1024**3
    # self.x = cp.ones(self.N)
    BT = np.array([[comb(ni, ki) for ni in range(n)] for ki in range(k+2)]).astype(np.int64)


    if not gpu:
      self.mult = np.multiply
      self.deg = np.ones(self.N, dtype=np.float32) * (n - k + 1)
      self.BT = BT
      if k == 3:
        self.launch_config = laplacian1_matvec
      elif k == 4:
        self.launch_config = laplacian2_matvec
      elif k == 5:
        self.launch_config = laplacian3_matvec
      elif k == 6:
        self.launch_config = laplacian4_matvec
      elif k == 7:
        self.launch_config = laplacian5_matvec
      else:
        raise ValueError("invalid k")
    else:
      self.mult = cp.multiply
      self.deg = cp.ones(self.N, dtype=np.float32) * (n - k + 1)
      self.BT = cp.array(BT)
      self.threadsperblock = threadsperblock
      self.blockspergrid = ((self.M + (self.threadsperblock - 1)) // threadsperblock) + 1
      self.n_kernels = n_kernels
      if k == 3:
        self.launch_config = laplacian1_matvec_cuda[self.blockspergrid, self.threadsperblock]
      elif k == 4:
        self.launch_config = laplacian2_matvec_cuda[self.blockspergrid, self.threadsperblock]
      elif k == 5:
        self.launch_config = laplacian3_matvec_cuda[self.blockspergrid, self.threadsperblock]
      elif k == 6:
        self.launch_config = laplacian4_matvec_cuda[self.blockspergrid, self.threadsperblock]
      elif k == 7:
        self.launch_config = laplacian5_matvec_cuda[self.blockspergrid, self.threadsperblock]
      else:
        raise ValueError("invalid k")

  def __repr__(self) -> str:
    msg = f"Up-Laplacian (({self.N} / {self.M})  ({self.k-1}/{self.k})-simplices) \n"
    msg += f"n: {self.n}, k: {self.k}, deg: {self.deg[:2]}\n"
    msg += (f"Pred memory usage cap: {self._pred_GB:.3f} GB \n")
    if hasattr(self, "threadsperblock"):
      msg += f"threads-per-block: {self.threadsperblock}, blocks-per-thread: {self.blockspergrid}\n"
    return msg

  def __call__(self, x: np.ndarray, y: np.ndarray, offset: int = 0) -> np.ndarray:
    assert len(x) == len(self.deg) and len(y) == len(self.deg), "Invalid dimensions"
    self.mult(x, self.deg, y)
    self.launch_config(x, y, self.n, self.k, self.M, self.BT, self.deg)
    return y

In [ ]:
L = LaplacianBenchmark(16, 4, gpu=True, n_kernels=1)
print(L)
# print(L.tm)
x = L.tm.arange(L.N, dtype=np.float32)
y = L.tm.zeros(L.N, dtype=np.float32)
# L(x,y)
# WUT = y.copy()

In [ ]:
n,k = 512, 5

GB_xyd = 3 * ((comb(n,k-1) * 4) / 1024**3)
GB_BT = (n*(k+2) * 8) / 1024**3
print(f"Predicted cap memory usage: {(GB_xyd + GB_BT):.3f} GB (xyd: {GB_xyd:.3f}, BT: {GB_BT:.3f})")

In [ ]:
y[0]

In [ ]:
np.all(y == WUT)

In [ ]:
import subprocess
subprocess.Popen("nvprof")

In [ ]:
import timeit

timings = {}
for n in [16,32,64,128,256,512,1024,2048]:
  for k in [3,4,5,6,7]:
    if predict_GB(n,k) > 12:
      continue
    L = LaplacianBenchmark(n, k, gpu=True)
    x = L.tm.arange(L.N)
    y = L.tm.zeros(L.N)
    print((n,k))
    timings[(n,k)] = timeit.repeat(lambda: L(x,y), number=1, repeat=30)
  print(n)


In [ ]:
# timings

In [ ]:
L = LaplacianBenchmark(2056, 3, gpu=True, threadsperblock=32)
x = L.tm.arange(L.N)
y = L.tm.zeros(L.N)
L(x,y)
## 44 seconds for 1024, 4

In [ ]:
n,k

In [ ]:

from json import dumps, dump
# s = dumps({str(k): v for k, v in timings.items()})

timings_ = { str(k) : v for k,v in timings.items()}
with open('timings.json', 'w') as fp:
    dump(timings_, fp)
# np.save

In [ ]:
threadsperblock = 32
blockspergrid = (M + (threadsperblock - 1))
print(f"# blocks: {blockspergrid}, threads per block: {threadsperblock}")
if k == 3:
  print(laplacian1_matvec_cuda)
  launch_config = laplacian1_matvec_cuda[blockspergrid, threadsperblock]
elif k == 4:
  print(laplacian2_matvec_cuda)
  launch_config = laplacian2_matvec_cuda[blockspergrid, threadsperblock]
elif k == 5:
  print(laplacian3_matvec_cuda)
  launch_config = laplacian3_matvec_cuda[blockspergrid, threadsperblock]
elif k == 6:
  print(laplacian4_matvec_cuda)
  launch_config = laplacian4_matvec_cuda[blockspergrid, threadsperblock]
elif k == 7:
  print(laplacian5_matvec_cuda)
  launch_config = laplacian5_matvec_cuda[blockspergrid, threadsperblock]
else:
  print("INVALID CONFIG")
print(launch_config)

In [ ]:
import cupy as cp
# x = cp.ones(comb(n,k-1))
x = cp.arange(comb(n,k-1))
y = cp.zeros(comb(n,k-1))
deg = cp.ones(comb(n,k-1)) * (n - k + 1) #cp.repeat(n - k + 1, comb(n, k-1))

print(f"n: {n}, k: {k}, deg: {deg[:5]}, x: {x[:5]}, y: {y[:5]}")
print(f"tpb: {threadsperblock}, bpg: {blockspergrid}")

In [ ]:
cp.multiply(x, deg, y)
print(y)
launch_config(x, y, n, k, int(comb(n, k)), BT, deg)
print(y)

In [ ]:
# mempool = cp.get_default_memory_pool()
# mempool.free_all_blocks()

In [ ]:
# deg = cp.array(precompute_deg(n,k,N,M,BT))
x = cp.arange(N)
y = cp.zeros(N)
BT = cp.array(np.array([[comb(ni, ki) for ni in range(n)] for ki in range(k+2)]).astype(np.int64))
deg = cp.ones(comb(n,k-1)) * (n - k + 1)

threadsperblock = 32
blockspergrid = (M + (threadsperblock - 1))
print(f"n: {n}, k: {k}, deg: {deg[:5]}, x: {x[:5]}, y: {y[:5]}")
print(f"tpb: {threadsperblock}, bpg: {blockspergrid}")

In [ ]:
cp.multiply(x, deg, y)
launch_config(x, y, n, k, int(comb(n, k)), BT, deg)

In [ ]:
# print(np.all(y == 0))
print(y)

In [ ]:
L = LaplacianBenchmark(64, 7, gpu=True)
# print(L.N)
# wut = cp.zeros(10) # 64, 6
L

In [ ]:
L = LaplacianBenchmark(64, 7, gpu=True)

x = L.tm.arange(L.N, dtype=np.float32)
y = L.tm.zeros(L.N, dtype=np.float32)
L(x,y)

In [ ]:
1.5300000e+02, 5.0000000e+02, 5.4500000e+02, ..., 3.6973199e+09,
3.8342625e+09, 3.9712087e+09

In [ ]:
L = LaplacianBenchmark(64, 6, gpu=False)
x = L.tm.arange(L.N)
y = L.tm.zeros(L.N)
L(x,y)

In [ ]:
print(timings)

In [ ]:
import timeit


timings = {}
for n in [16, 32, 64, 128]:
  for k in [4,5,6,7]:
    L = LaplacianBenchmark(n, k)
    x = cp.arange(L.N, dtype=np.float32)
    y = cp.zeros(L.N, dtype=np.float32)
    print((n,k))
    timings[(n,k)] = timeit.timeit(lambda: L(x,y), number=100)
  print(n)

# 6.  10.  14.  14.  18.   8.  12.   8.  12.  28.  20.  32.   7.  19.
#   37. -24. -20. -28. -24.  -4. -16.  -4. -35. -23.  -5.  48.  20.  52.
#  -13.  19.  57. -55. -23.  15.  63.
# launch_config(x, y, n, k, N, BT, deg)

# BT_cuda = cp.array(BT)
# def laplacian_matvec_cuda() -> np.ndarray:
#   cp.multiply(x, deg, y)
#   launch_config(x, y, n, k, N, BT_cuda, deg)
#   return y

# import timeit

In [ ]:
# timeit.timeit(lambda: laplacian_matvec_cuda(), number=100)

In [ ]:
# from cupyx.profiler import benchmark

# benchmark(laplacian_matvec_cuda, n_repeat=100)

In [ ]:
from numba import int64, float64

@cuda.jit
def test_cuda(N: int, out: np.ndarray):
  tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
  ps = cuda.local.array(shape=(8,), dtype=int64)
  if tid < N:
    out[tid] = tid
  # out[0] = tid if tid > out[0] else out[0]
  cuda.atomic.max(out, 0, tid)

In [ ]:
import cupy as cp
out = cp.zeros(512)
threadsperblock = 32
blockspergrid = 512 // 32
print(f"# blocks: {blockspergrid}, threads per block: {threadsperblock}")
test_cuda[blockspergrid, threadsperblock](len(out), out)

In [ ]:
print(min(out))
print(out[0])
print(out)

In [ ]:
2**16